In [ ]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.7 MB/s eta 0:00:00


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import tensorboard
import random
from netCDF4 import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torch.distributions import Normal, Dirichlet


# **Radius model**

In [ ]:
class CustomDistributionLambdaWithRegularizer(nn.Module):
    def __init__(self, make_distribution_fn, prior):
        super(CustomDistributionLambdaWithRegularizer, self).__init__()
        self.make_distribution_fn = make_distribution_fn
        self.prior = prior

    def forward(self, inputs):
        distribution = self.make_distribution_fn(inputs)
        kl_divergence = torch.distributions.kl.kl_divergence(distribution, self.prior).mean()
        return distribution, kl_divergence

In [ ]:
class CustomDistributionLambda(nn.Module):
    def __init__(self, make_distribution_fn):
        super(CustomDistributionLambda, self).__init__()
        self.make_distribution_fn = make_distribution_fn

    def forward(self, inputs):
        return self.make_distribution_fn(inputs)

In [ ]:
def make_gamma_distribution(inputs):
    concentration = inputs[..., 0]
    rate = inputs[..., 1]
    return torch.distributions.gamma.Gamma(concentration, rate)

In [ ]:
class U_Encoder(nn.Module):
    def __init__(self):
        super(U_Encoder, self).__init__()
        self.alpha = 1.5
        self.prior = self.make_mvn_prior(1, True)
        self.dense1 = nn.Linear(1, 5)
        self.dense1.weight.data.uniform_(0.01, 0.02)
        self.dense2 = nn.Linear(5, 5)
        self.dense2.weight.data.uniform_(0.01, 0.02)
        self.dense3 = nn.Linear(5, 2)
        self.dense3.bias.data.uniform_(3.0, 5.5)
        self.lambda1 = lambda x: torch.abs(x) + 0.001
        self.dist_lambda3 = CustomDistributionLambdaWithRegularizer(make_distribution_fn=make_gamma_distribution, prior=self.prior)

    def make_mvn_prior(self, ndim, Trainable=True):
        if Trainable:
            c = nn.Parameter(torch.rand([ndim]) * (5.5 - 3.5) + 3.5)
            rate = 1
        else:
            c = self.alpha
            rate = 1
        prior = torch.distributions.gamma.Gamma(concentration=c, rate=rate)
        return prior

    def forward(self, inputs):
        x = F.relu(self.dense1(inputs))
        x = F.relu(self.dense2(x))
        x = F.relu(self.dense3(x))
        x = self.lambda1(x)
        gamma_dist = self.dist_lambda3(x)
        pdf_at_x = gamma_dist[0].log_prob(x)
        return pdf_at_x.exp()

class U_Decoder(nn.Module):
    def __init__(self):
        super(U_Decoder, self).__init__()
        self.dense1 = nn.Linear(1, 5)
        self.lambda1 = lambda x: 1 / x
        self.dense2 = nn.Linear(5, 5)
        self.dense31 = nn.Linear(5, 1)
        self.dense32 = nn.Linear(5, 1)
        self.lambda2 = lambda x: torch.abs(x) + 0.001
        self.dist_lambda1 = CustomDistributionLambda(make_distribution_fn=make_gamma_distribution)

    def forward(self, inputs):
        y1 = inputs.view(-1, 1)
        y = self.lambda1(y1)
        x = F.relu(self.dense1(y))
        x = F.relu(self.dense2(x))
        alpha = self.dense31(x)
        alpha = self.lambda2(alpha * y1)
        beta = self.dense32(x)
        beta = self.lambda2(beta / y**2)
        x = torch.cat([alpha, beta], dim=1)
        gamma_dist = self.dist_lambda1(x)
        pdf_at_x = gamma_dist.log_prob(x)
        return pdf_at_x.exp()

class U_Ext_VAE(nn.Module):
    def __init__(self):
        super(U_Ext_VAE, self).__init__()
        self.encoder = U_Encoder()
        self.decoder = U_Decoder()

    def forward(self, inputs):
        res = self.decoder(self.encoder(inputs))
        return res

In [ ]:
ncfile = Dataset('/content/Dataset_danube.nc',"r")
L=[]
for i in range(31):
    L.append(ncfile['S'+str(i+1)][:].reshape(18244,1))

dataset = np.concatenate((L[0],L[1],L[2],L[3],L[4],L[5],L[6],L[7],L[8],L[9],L[10],L[11],L[12],L[13],L[14],L[15],L[16],L[17],L[18],L[19],L[20],L[21],L[22],L[23],L[24],L[25],L[26],L[27],L[28],L[29],L[30]),axis=1)

In [ ]:
R4 = dataset[:,22:27]
axis = 1
R4_rad = np.sum(R4,axis)
R4_rad = R4_rad.reshape((18244,1))
R4_S = np.divide(R4,R4_rad)
train_dataset = R4_rad[::25,:]
eval_dataset  = R4_rad
print(train_dataset.shape)
print(np.max(train_dataset))
print(np.max(R4_rad))

(730, 1)
856.9
1899.0


In [ ]:
def negative_log_likelihood(x, rv_x):
  mean = rv_x[:, 0]
  log_variance = rv_x[:, 1]
  dist = Normal(mean, torch.exp(0.5 * log_variance))  # Assuming log_variance represents the log-variance
  log_prob = -dist.log_prob(x).mean()  # Calculate the negative log-likelihood

  return log_prob

In [ ]:
# Initialize the model
vae = U_Ext_VAE()

# Define optimizer and loss function
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

train_loader = train_dataset
eval_loader = train_dataset

# Training loop
num_epochs = 5000
batch_size = 32


for epoch in range(num_epochs):
    vae.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} (Training)"):
        optimizer.zero_grad()

        masked_array = batch
        numpy_array = masked_array.data
        inputs = torch.from_numpy(numpy_array).float()

        reconstructed_output = vae(inputs)
        reconstructed_output = torch.where(torch.isnan(reconstructed_output), torch.zeros_like(reconstructed_output), reconstructed_output)
        reconstructed_output[torch.isnan(reconstructed_output)] = 0
        loss = negative_log_likelihood(inputs, reconstructed_output)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    vae.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(eval_loader, desc=f"Epoch {epoch + 1}/{num_epochs} (Validation)"):
            inputs = batch
            masked_array = batch
            numpy_array = masked_array.data
            inputs = torch.from_numpy(numpy_array).float()
            reconstructed_output = vae(inputs)
            reconstructed_output = torch.where(torch.isnan(reconstructed_output), torch.zeros_like(reconstructed_output), reconstructed_output)
            reconstructed_output[torch.isnan(reconstructed_output)] = 0
            loss = negative_log_likelihood(inputs, reconstructed_output)
            eval_loss += loss.item()

    # Calculate average loss
    train_loss /= len(train_loader)
    eval_loss /= len(eval_loader)

    # Print progress
    print(f"Epoch [{epoch + 1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.4f} | Eval Loss: {eval_loss:.4f}")


Epoch 1/5000 (Validation): 100%|██████████| 730/730 [00:00<00:00, 735.02it/s]


Epoch [1/5000] | Train Loss: 8.2143 | Eval Loss: 6.7989


Epoch 2/5000 (Validation): 100%|██████████| 730/730 [00:00<00:00, 937.71it/s]


Epoch [2/5000] | Train Loss: 6.8302 | Eval Loss: 6.8057


Epoch 3/5000 (Validation): 100%|██████████| 730/730 [00:00<00:00, 909.79it/s]


...

Epoch [413/5000] | Train Loss: 6.4266 | Eval Loss: 6.4401


Epoch 414/5000 (Validation): 100%|██████████| 730/730 [00:00<00:00, 921.21it/s]


Epoch [414/5000] | Train Loss: 6.4266 | Eval Loss: 6.4401


Epoch 415/5000 (Training):  35%|███▍      | 253/730 [00:00<00:01, 266.34it/s]


KeyboardInterrupt: ignored

In [ ]:
class IndependentNormal(nn.Module):
    def __init__(self, num_features):
        super(IndependentNormal, self).__init__()
        self.num_features = num_features
        self.locs = nn.Linear(num_features, num_features)
        self.scales = nn.Linear(num_features, num_features)

    def forward(self, x):
        locs = self.locs(x)
        scales = F.softplus(self.scales(x)) + 1e-5  # Adding a small value to prevent numerical issues
        return Normal(locs, scales)

In [ ]:
class IndependentDirichlet(nn.Module):
    def __init__(self, num_features):
        super(IndependentDirichlet, self).__init__()
        self.num_features = num_features
        self.alphas = nn.Linear(num_features, num_features)

    def forward(self, x):
        alphas = F.softplus(self.alphas(x)) + 1e-5  # Adding a small value to prevent numerical issues
        return Dirichlet(alphas)

In [ ]:
import torch.distributions as td

class Sphere_Encoder(nn.Module):
    def __init__(self):
        super(Sphere_Encoder, self).__init__()
        self.encoded_size = 4
        self.prior = td.Independent(td.Normal(loc=torch.zeros(self.encoded_size), scale=torch.ones(self.encoded_size)),
                                    reinterpreted_batch_ndims=1)
        self.concat = nn.Linear(8, 8)
        self.dense1 = nn.Linear(6, 8)
        self.dense2 = nn.Linear(8, 8)
        self.lambda1 = lambda x: torch.abs(x) + 0.001
        self.dense3 = nn.Linear(8, self.encoded_size)
        self.ind_norm1 = IndependentNormal(self.encoded_size)

    def forward(self, inputs):
        inputs1, inputs2 = inputs
        x = torch.cat([inputs1, inputs2 / 10], dim=1)
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = torch.abs(x) + 0.001
        x = self.dense3(x)
        x = self.ind_norm1(x)
        return x

class Sphere_Decoder(nn.Module):
    def __init__(self):
        super(Sphere_Decoder, self).__init__()
        self.K = 5
        self.dense1 = nn.Linear(4, 5)
        self.dense2 = nn.Linear(6, 10)
        self.dense31 = nn.Linear(self.K * 2, self.K)
        self.ind_norm11 = IndependentNormal(self.K)
        self.dense32 = nn.Linear(5, 5)
        self.lambda1 = lambda x: 1 / (1 + x)
        self.dirichlet = IndependentDirichlet(self.K)

    def forward(self, inputs):
        inputs1, inputs2 = inputs
        inputs2 = self.lambda1(inputs2)
        print(inputs1)
        x = F.relu(self.dense1(inputs1.mean))
        x = torch.cat([x, inputs2], dim=1)
        x = F.relu(self.dense2(x))
        x = self.dense31(x)
        x = self.ind_norm11(x)
        x = self.dirichlet(x.mean)
        return x.mean

class Sphere_VAE(nn.Module):
    def __init__(self):
        super(Sphere_VAE, self).__init__()
        self.encoder = Sphere_Encoder()
        self.decoder = Sphere_Decoder()

    def forward(self, inputs):
        inputs1, inputs2 = inputs
        return self.decoder([self.encoder([inputs1, inputs2]), inputs2])

In [ ]:
angle_train_dep = R4_S[::25,:]
print(angle_train_dep.shape)
print(train_dataset.shape)
angle_eval_dep  = R4_S
print(angle_eval_dep.shape)

(730, 5)
(730, 1)
(18244, 5)


In [ ]:
ang_vae = Sphere_VAE()

# Define optimizer and loss function
optimizer = optim.Adam(ang_vae.parameters(), lr=1e-3)

train_loader = train_dataset
eval_loader = train_dataset


epochs = 5000
batch_size = 32

for epoch in range(epochs):
    epoch_loss = 0.0
<r
    # Training
    ang_vae.train()
    with tqdm(range(0, len(angle_train_dep), batch_size)) as t:
        for i in t:
            optimizer.zero_grad()
            angle_train_batch = angle_train_dep[i:i+batch_size]
            train_data_batch = train_dataset[i:i+batch_size]
            print(angle_train_batch.shape)
            print(train_data_batch.shape)
            angle_train_batch = torch.tensor(angle_train_batch, dtype=torch.float32)
            train_data_batch = torch.tensor(train_data_batch, dtype=torch.float32)

            output = ang_vae([angle_train_batch, train_data_batch])
            print(output.shape)
            loss = negative_log_likelihood(output, angle_train_batch)  # Change this line

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            t.set_postfix(loss=epoch_loss / (i / batch_size + 1))  # Update progress bar with loss

    epoch_loss /= len(angle_train_dep) / batch_size

    # Validation
    ang_vae.eval()
    val_loss = 0.0
    with torch.no_grad(), tqdm(range(0, len(angle_eval_dep), batch_size)) as t:
        for i in t:
            angle_val_batch = angle_eval_dep[i:i+batch_size]
            eval_data_batch = eval_dataset[i:i+batch_size]

            angle_val_batch = torch.tensor(angle_val_batch, dtype=torch.float32)
            eval_data_batch = torch.tensor(eval_data_batch, dtype=torch.float32)

            output = ang_vae([angle_val_batch, eval_data_batch])

            loss = negative_log_likelihood(output, angle_val_batch)  # Change this line

            val_loss += loss.item()
            t.set_postfix(val_loss=val_loss / (i / batch_size + 1))  # Update progress bar with validation loss

    val_loss /= len(angle_eval_dep) / batch_size

    print(f"Epoch [{epoch+1}/{epochs}] Loss: {epoch_loss:.4f} Val Loss: {val_loss:.4f}")